        __  ______ _       ________  __        _ 
       / / / / __ \ |     / / __ \ \/ /  _    | |
      / /_/ / / / / | /| / / / / /\  /  (_)   / /
     / __  / /_/ /| |/ |/ / /_/ / / /  _     / / 
    /_/ /_/\____/ |__/|__/_____/ /_/  (_)  _/_/  
                                          /_/    
                                                      
               5168 7450 1701 5535     

                 author = Valerij Syroiezhin
               telegram = @NEU3RON
                  email = v.syroiezhin@gmail.com
                    url = https://github.com/syroiezhin

# 🏗 Groundwork 🪚

In [75]:
from pandas import ExcelWriter, DataFrame, ExcelFile, merge, concat, read_excel
from openpyxl import load_workbook
from os import system, remove
from os.path import isfile
from glob import glob

xlsx = 'unite.xlsx'
chief = 'chief'
monthly_finance = [ f'M{n}' for n in range(1,10) ]

# 👷🏾‍♂️ Functional 👨🏽‍🏭

In [76]:
def newfile(name:str): 
    if isfile(xlsx): remove(xlsx)
    ExcelWriter(name).save()

def capacity(typefile:str)->list: return sorted([ file for file in glob(f'*.{typefile}') if '~$' not in file ])

def fill(file:str, namesheet:str, content:list, columns:list=None, mode:str='w', index:bool=False, header:bool=True):
    with ExcelWriter(file, mode=mode, engine="openpyxl", if_sheet_exists=(None if mode=='w' else 'overlay')) as writer:  
        content.to_excel(writer, sheet_name=namesheet, index=index, header=header, columns=columns)
        
def unitexls(fxls:list):
    list_sheet=[]
    for fxl in fxls:
        ef = ExcelFile(fxl)
        print('\n', fxl, ":", end=' ')
        for sheet in ef.sheet_names:
            list_sheet.append(sheet) if sheet not in list_sheet else list_sheet.append(sheet+'_'+fxl.replace('.xlsx',''))
            fill(content=read_excel(ef, sheet_name=sheet), file=xlsx, namesheet=list_sheet[-1], mode='a', header=True)
            print(list_sheet[-1], end=' ')

def rename(file:str, sheet:str, oldnamesheet:str='Sheet1'):
    fxl = load_workbook(file)
    fxl[oldnamesheet].title = sheet
    fxl.save(file)

def check(file:str, message:str='\nResult of checking:'):
    fxl = load_workbook(file)
    sheets = fxl.sheetnames
    print(message, sheets, end='\n\n')

def scan(file:str, namesheet:str, start:str='A', upto:str='A1') -> DataFrame:
    tabl = []
    fxl = load_workbook(file)
    cells = fxl[namesheet][start+'1':upto]
    columns = [ cells[0][c].value for c in range(len(cells[0])) ]
    for cell in cells[1:]: tabl.append([ cell[c].value for c in range(len(cell)) ])
    return DataFrame( tabl, columns=columns, index=list(range(1,int(upto[1:]))) )

def connect(dfs:DataFrame) -> DataFrame:
    for df in dfs[1:]: dfs[0] = dfs[0].join(df,how='right')
    return dfs[0]

def add2matrix(matrix:list, index:int, value:list): matrix[index]=matrix[index]+value

def column2rows(df:DataFrame, column:int) -> list:
    D=[[]]*column
    enum = -1
    for k,v in df.values: 
        if int(k)==1: enum+=1
        add2matrix(D,enum,[v])
    return D

def enable(file:str, turnoff:str="Microsoft Excel", afterseconds:int=60): system(f'''open {file}; sleep {afterseconds};   killall "{turnoff}" || taskkill /f /im {turnoff.split()[-1]}.exe /t''') # mac || win

# 🫡 Screenplay 📝

In [77]:
newfile(xlsx)
fxls = capacity('xlsx')
fxls.remove(xlsx)
unitexls(fxls)
rename(xlsx, chief)
check(xlsx)


 ID.xlsx : ID 
 packet.xlsx : packet 
 payment.xlsx : payment 
Result of checking: ['chief', 'ID', 'packet', 'payment']



In [78]:
# we work in file payment
dfme = scan(file=xlsx, namesheet='payment', upto='A87')
dfmp = scan(file=xlsx, namesheet='payment', upto='B87', start='B')
df = concat([ connect( [dfme,dfmp] ) ], ignore_index=True).drop_duplicates(ignore_index=True)
print(df)

    id  packet
0  109       3
1  109       4
2  105       3
3  105       4
4   94       3
5   94       4
6  132       3
7  132       4
8   81       3
9   81       4


In [79]:
# we work in file packet
dfp = scan(file=xlsx, namesheet='packet', upto='B6')
dfp = DataFrame([ nm for d in df['packet'] for id,nm in dict(dfp.values).items() if d==id ], columns=['packet_name'])
df = connect( [df,dfp] )
print(df)

    id  packet           packet_name
0  109       3   Хірургічна допомога
1  109       4  Стаціонарна допомога
2  105       3   Хірургічна допомога
3  105       4  Стаціонарна допомога
4   94       3   Хірургічна допомога
5   94       4  Стаціонарна допомога
6  132       3   Хірургічна допомога
7  132       4  Стаціонарна допомога
8   81       3   Хірургічна допомога
9   81       4  Стаціонарна допомога


In [80]:
# we work in file ID
dfef = scan(file=xlsx, namesheet='ID', upto='B24')
dfee = scan(file=xlsx, namesheet='ID', upto='C24', start='C')
dfee.rename(columns={'ID':'id'}, inplace = True)
dfen = scan(file=xlsx, namesheet='ID', upto='D24', start='D')
dfef = connect( [dfee,dfen,dfef] )
df = merge(df,dfef,on='id').drop_duplicates(subset=['id','packet'],ignore_index=True)
print(df)

    id  packet           packet_name Назва закладу Область реєстрації  \
0  109       3   Хірургічна допомога             w             М.КИЇВ   
1  109       4  Стаціонарна допомога             w             М.КИЇВ   
2  105       3   Хірургічна допомога             v             М.КИЇВ   
3  105       4  Стаціонарна допомога             v             М.КИЇВ   
4   94       3   Хірургічна допомога             l   ДНІПРОПЕТРОВСЬКА   
5   94       4  Стаціонарна допомога             l   ДНІПРОПЕТРОВСЬКА   
6  132       3   Хірургічна допомога             t         РІВНЕНСЬКА   
7  132       4  Стаціонарна допомога             t         РІВНЕНСЬКА   
8   81       3   Хірургічна допомога             s         ПОЛТАВСЬКА   
9   81       4  Стаціонарна допомога             s         ПОЛТАВСЬКА   

  Форма власності  
0      Комунальна  
1      Комунальна  
2      Комунальна  
3      Комунальна  
4      Комунальна  
5      Комунальна  
6      Комунальна  
7      Комунальна  
8      Комунальн

In [81]:
# add cash costs from payment file
dfmm = scan(file=xlsx, namesheet='payment', upto='D87', start='C')
dfmm = DataFrame( column2rows(dfmm,10), columns=monthly_finance )
df = connect( [df,dfmm] )
print(df, end='\n\n')

    id  packet           packet_name Назва закладу Область реєстрації  \
0  109       3   Хірургічна допомога             w             М.КИЇВ   
1  109       4  Стаціонарна допомога             w             М.КИЇВ   
2  105       3   Хірургічна допомога             v             М.КИЇВ   
3  105       4  Стаціонарна допомога             v             М.КИЇВ   
4   94       3   Хірургічна допомога             l   ДНІПРОПЕТРОВСЬКА   
5   94       4  Стаціонарна допомога             l   ДНІПРОПЕТРОВСЬКА   
6  132       3   Хірургічна допомога             t         РІВНЕНСЬКА   
7  132       4  Стаціонарна допомога             t         РІВНЕНСЬКА   
8   81       3   Хірургічна допомога             s         ПОЛТАВСЬКА   
9   81       4  Стаціонарна допомога             s         ПОЛТАВСЬКА   

  Форма власності       M1        M2        M3        M4        M5        M6  \
0      Комунальна  9878.67  11190.24  11190.24  11190.24  11190.24  11190.24   
1      Комунальна  4561.31   4776.89

In [ ]:
# write everything in xlsx file and rejoice
fill(xlsx, chief, df, mode='a')
enable(xlsx)

# 🐼 Using only pandas

In [67]:
import pandas as pd
pd.ExcelFile('unite.xlsx').sheet_names

['chief', 'ID', 'packet', 'payment']

In [68]:
payments = pd.read_excel('unite.xlsx', sheet_name='payment')
payments

,id,packet,month,sum
0,109,3,1,9878.67
1,109,3,2,11190.24
2,109,3,3,11190.24
3,109,3,4,11190.24
4,109,3,5,11190.24
...,...,...,...,...
81,81,4,4,761.74
82,81,4,5,761.74
83,81,4,6,761.74
84,81,4,7,916.34


In [69]:
payments = payments.pivot_table(index=['id' ,'packet'], columns='month', values='sum').reset_index()
payments

month,id,packet,1,2,3,4,5,6,7,8,9
0,81,3,1636.77,2002.42,2002.42,2002.42,2002.42,2002.42,1959.41,1401.69,NaN
1,81,4,642.25,761.74,761.74,761.74,761.74,761.74,916.34,533.22,NaN
2,94,3,357.81,384.80,384.80,384.80,384.80,384.80,335.51,269.36,NaN
3,94,4,872.14,896.29,896.29,896.29,896.29,896.29,884.98,627.40,NaN
4,105,3,13.09,13.19,13.19,13.19,13.19,13.19,9.23,9.23,9.23
5,105,4,140.75,205.58,205.58,205.58,205.58,205.58,193.88,143.91,143.91
6,109,3,9878.67,11190.24,11190.24,11190.24,11190.24,11190.24,11006.49,7833.17,7833.17
7,109,4,4561.31,4776.89,4776.89,4776.89,4776.89,4776.89,4571.74,3343.82,3343.82
8,132,3,411.88,470.67,470.67,470.67,470.67,470.67,451.86,329.46,329.46
9,132,4,630.31,681.19,681.19,681.19,681.19,681.19,848.43,476.83,476.83


In [70]:
packets = pd.read_excel('unite.xlsx', sheet_name='packet')
packets


,packet,packet_name
0,1,Первинна допомога
1,2,Екстрена допомога
2,3,Хірургічна допомога
3,4,Стаціонарна допомога
4,5,Медична допомога при інсульті


In [71]:
packets = packets.set_index('packet').to_dict()['packet_name']
packets

{1: 'Первинна допомога',
 2: 'Екстрена допомога',
 3: 'Хірургічна допомога',
 4: 'Стаціонарна допомога',
 5: 'Медична допомога при інсульті'}

In [72]:
payments.insert(2, 'name', payments['packet'].map(packets))
payments

month,id,packet,name,1,2,3,4,5,6,7,8,9
0,81,3,Хірургічна допомога,1636.77,2002.42,2002.42,2002.42,2002.42,2002.42,1959.41,1401.69,NaN
1,81,4,Стаціонарна допомога,642.25,761.74,761.74,761.74,761.74,761.74,916.34,533.22,NaN
2,94,3,Хірургічна допомога,357.81,384.80,384.80,384.80,384.80,384.80,335.51,269.36,NaN
3,94,4,Стаціонарна допомога,872.14,896.29,896.29,896.29,896.29,896.29,884.98,627.40,NaN
4,105,3,Хірургічна допомога,13.09,13.19,13.19,13.19,13.19,13.19,9.23,9.23,9.23
5,105,4,Стаціонарна допомога,140.75,205.58,205.58,205.58,205.58,205.58,193.88,143.91,143.91
6,109,3,Хірургічна допомога,9878.67,11190.24,11190.24,11190.24,11190.24,11190.24,11006.49,7833.17,7833.17
7,109,4,Стаціонарна допомога,4561.31,4776.89,4776.89,4776.89,4776.89,4776.89,4571.74,3343.82,3343.82
8,132,3,Хірургічна допомога,411.88,470.67,470.67,470.67,470.67,470.67,451.86,329.46,329.46
9,132,4,Стаціонарна допомога,630.31,681.19,681.19,681.19,681.19,681.19,848.43,476.83,476.83


# 🎁 Using SQL

In [73]:
from pandasql import sqldf

try:
    sqldf('select id,  from payments')
except Exception as e: 
    print(e)

(sqlite3.OperationalError) near "from": syntax error
[SQL: select id,  from payments]
(Background on this error at: https://sqlalche.me/e/14/e3q8)
